In [2]:
# Import
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor

In [3]:
data, meta = arff.loadarff('../../Data/Regression/fps in video games/fps-in-video-games.arff')
df = pd.DataFrame(data)
df

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,...,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,b'Intel Core i7-920',4.0,8.0,133.0,256.0,1024.0,8.0,0.000263,2666.0,20.0,...,2304.0,144.0,182300.0,5700.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'low',b'userbenchmark',70.0
1,b'Intel Core i7-920',4.0,8.0,133.0,256.0,1024.0,8.0,0.000263,2666.0,20.0,...,2304.0,144.0,182300.0,5700.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'low',b'userbenchmark',230.0
2,b'Intel Core i5-4690',4.0,4.0,100.0,256.0,1024.0,6.0,0.000177,3400.0,34.0,...,2560.0,160.0,277300.0,7200.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'low',b'userbenchmark',250.0
3,b'Intel Core i5-4690',4.0,4.0,100.0,256.0,1024.0,6.0,0.000177,3400.0,34.0,...,2560.0,160.0,277300.0,7200.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'low',b'userbenchmark',300.0
4,b'Intel Core i5-4690',4.0,4.0,100.0,256.0,1024.0,6.0,0.000177,3400.0,34.0,...,896.0,56.0,61600.0,2080.0,b'1.2.131',b'counterStrikeGlobalOffensive',1080.0,b'low',b'userbenchmark',200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425828,b'AMD Ryzen 5 3600',6.0,12.0,100.0,576.0,3072.0,32.0,0.000074,3600.0,36.0,...,2048.0,128.0,159200.0,5700.0,b'1.2.131',b'rainbowSixSiege',1080.0,b'max',b'fpsbenchmark',51.5
425829,b'AMD Ryzen 5 2600X',6.0,12.0,100.0,576.0,3072.0,16.0,0.000192,3600.0,36.0,...,2048.0,128.0,159200.0,5700.0,b'1.2.131',b'rainbowSixSiege',1080.0,b'max',b'fpsbenchmark',49.9
425830,b'AMD Ryzen 5 1600X',6.0,12.0,100.0,576.0,3072.0,16.0,0.000192,3600.0,36.0,...,2048.0,128.0,159200.0,5700.0,b'1.2.131',b'rainbowSixSiege',1080.0,b'max',b'fpsbenchmark',46.9
425831,b'AMD Ryzen 5 2600',6.0,12.0,100.0,576.0,3072.0,16.0,0.000192,3400.0,34.0,...,2048.0,128.0,159200.0,5700.0,b'1.2.131',b'rainbowSixSiege',1080.0,b'max',b'fpsbenchmark',46.4


In [4]:
for col in df.columns:
    if df[col].dtype == 'object':  # Vérifie si le type de la colonne est object, souvent utilisé pour les strings
        df[col] = df[col].apply(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)
df

,CpuName,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuDieSize,CpuFrequency,CpuMultiplier,...,GpuNumberOfShadingUnits,GpuNumberOfTMUs,GpuTextureRate,GpuNumberOfTransistors,GpuVulkan,GameName,GameResolution,GameSetting,Dataset,FPS
0,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,0.000263,2666.0,20.0,...,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,70.0
1,Intel Core i7-920,4.0,8.0,133.0,256.0,1024.0,8.0,0.000263,2666.0,20.0,...,2304.0,144.0,182300.0,5700.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,230.0
2,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,0.000177,3400.0,34.0,...,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,250.0
3,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,0.000177,3400.0,34.0,...,2560.0,160.0,277300.0,7200.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,300.0
4,Intel Core i5-4690,4.0,4.0,100.0,256.0,1024.0,6.0,0.000177,3400.0,34.0,...,896.0,56.0,61600.0,2080.0,1.2.131,counterStrikeGlobalOffensive,1080.0,low,userbenchmark,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425828,AMD Ryzen 5 3600,6.0,12.0,100.0,576.0,3072.0,32.0,0.000074,3600.0,36.0,...,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,51.5
425829,AMD Ryzen 5 2600X,6.0,12.0,100.0,576.0,3072.0,16.0,0.000192,3600.0,36.0,...,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,49.9
425830,AMD Ryzen 5 1600X,6.0,12.0,100.0,576.0,3072.0,16.0,0.000192,3600.0,36.0,...,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,46.9
425831,AMD Ryzen 5 2600,6.0,12.0,100.0,576.0,3072.0,16.0,0.000192,3400.0,34.0,...,2048.0,128.0,159200.0,5700.0,1.2.131,rainbowSixSiege,1080.0,max,fpsbenchmark,46.4


In [9]:
df.dtypes.unique()

array([dtype('O'), dtype('float64')], dtype=object)

In [27]:
# Suppression des colonnes avec trop de valeurs manquante
seuil = 0.4
pourcentage_valeurs_manquantes = df.isnull().mean()
colonnes_a_supprimer = pourcentage_valeurs_manquantes[pourcentage_valeurs_manquantes > seuil].index
df = df.drop(columns=colonnes_a_supprimer)
print(f"Colonnes supprimées: {list(colonnes_a_supprimer)}")

Colonnes supprimées: ['CpuDieSize', 'CpuNumberOfTransistors', 'GpuNumberOfComputeUnits', 'GpuNumberOfExecutionUnits']


In [28]:
# # Supprimer les lignes avec au moins une valeur manquante
# df = df.dropna()
# df

In [29]:
def impute_with_decision_tree(df, target_column):
    # Séparer les lignes avec et sans valeurs manquantes pour la colonne cible
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df[~df[target_column].isnull()]

    # Vérifier s'il y a assez de données pour entraîner le modèle
    if df_not_missing.shape[0] < 10:
        return df

    # Appliquer get_dummies pour encoder les variables catégoriques
    df_encoded = pd.get_dummies(df, drop_first=True)

    # Réencodage des colonnes d'origine pour les utiliser dans la prédiction
    X = df_encoded.loc[df_not_missing.index].drop(columns=[target_column])
    y = df_not_missing[target_column]

    # Entraîner le modèle d'arbre de décision
    model = DecisionTreeRegressor()
    model.fit(X, y)

    # Imputer les valeurs manquantes
    X_missing = df_encoded.loc[df_missing.index].drop(columns=[target_column])
    df.loc[df[target_column].isnull(), target_column] = model.predict(X_missing)

    return df

# Appliquer l'imputation à toutes les colonnes avec des valeurs manquantes
for column in df.columns:
    if df[column].isnull().any():
        df = impute_with_decision_tree(df, column)

In [30]:
df.to_csv('../../Data/Regression/fps in video games/fps-in-video-games_clean.csv', index=False)

In [31]:
object_columns = df.select_dtypes(include=['object']).columns
df = pd.get_dummies(df, columns=object_columns)
df

,CpuNumberOfCores,CpuNumberOfThreads,CpuBaseClock,CpuCacheL1,CpuCacheL2,CpuCacheL3,CpuFrequency,CpuMultiplier,CpuProcessSize,CpuTDP,...,GameName_starcraft2,GameName_totalWar3Kingdoms,GameName_warframe,GameName_worldOfTanks,GameSetting_high,GameSetting_low,GameSetting_max,GameSetting_med,Dataset_fpsbenchmark,Dataset_userbenchmark
0,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,45.0,130.0,...,False,False,False,False,False,True,False,False,False,True
1,4.0,8.0,133.0,256.0,1024.0,8.0,2666.0,20.0,45.0,130.0,...,False,False,False,False,False,True,False,False,False,True
2,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,22.0,84.0,...,False,False,False,False,False,True,False,False,False,True
3,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,22.0,84.0,...,False,False,False,False,False,True,False,False,False,True
4,4.0,4.0,100.0,256.0,1024.0,6.0,3400.0,34.0,22.0,84.0,...,False,False,False,False,False,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425828,6.0,12.0,100.0,576.0,3072.0,32.0,3600.0,36.0,7.0,65.0,...,False,False,False,False,False,False,True,False,True,False
425829,6.0,12.0,100.0,576.0,3072.0,16.0,3600.0,36.0,12.0,95.0,...,False,False,False,False,False,False,True,False,True,False
425830,6.0,12.0,100.0,576.0,3072.0,16.0,3600.0,36.0,14.0,95.0,...,False,False,False,False,False,False,True,False,True,False
425831,6.0,12.0,100.0,576.0,3072.0,16.0,3400.0,34.0,12.0,65.0,...,False,False,False,False,False,False,True,False,True,False


In [32]:
# Créer le modèle de régression linéaire
model = XGBRegressor()

In [33]:
X = df.drop('FPS', axis=1)
y = df['FPS']
# Validation croisée
scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_absolute_error')

# Calculer la moyenne et l'écart-type des scores
mean_score = -scores.mean()  # Négatif car l'erreur absolue moyenne est une perte
std_score = scores.std()

print(f"Mean MAE: {mean_score}")
print(f"Standard Deviation of MAE: {std_score}")

Mean MAE: 40.90904265915431
Standard Deviation of MAE: 9.339257519601563
